# 类与对象(II)

* 属性与方法的安全性
* 类的特殊属性与特殊方法
* 类成员的访问拦截
* 对象的赋值与拷贝

## 方法和属性的安全性

通过类对数据和操作进行封装后，对象的数据和操作的细节就被屏蔽了，对于对象的数据一般通过调用其操作界面即对象的方法来实现。如何保证这些操作是安全的，是下一个问题。

### 私有方法和私有属性

python使用命名来规定私有属性/方法。与C++和Java不同（使用public和private关键字控制方法/属性的公有和私有）。

一般的，以**两个下划线开头但是不以两个下划线结尾的方法/属性为私有方法/属性。**

私有方法/属性只能在类内部访问，不能通过实例对象从外部访问。

In [1]:
class Shape:
    def __init__(self, name):
        self.name = name
        self.__area = 1  # 私有属性
    def get_area(self):
        return self.__area

In [2]:
s1 = Shape('circle')
#s1.__area  # error! why?
s1.get_area()

1

### @property装饰器

为了保证类中的数据成员的安全性，避免被外部程序无意修改，可以定义私有属性，然后定义相应的访问函数。

在访问函数中可以对属性设置进行校验，只有合乎规则的修改才能生效。

@property装饰器可以把函数装饰成属性，从而提供更加友好的访问方式。

In [3]:
class Point:
    def __init__(self, color):
        self.__color = color
    @property
    def color(self):
        print('access granted')
        return self.__color

In [4]:
t1 = Point('red')
t1.color

access granted


'red'

In [5]:
#t1.color = 'blue'  # can't set attribute

利用装饰器不仅可以实现私有属性的读取操作，也可以实现私有属性的修改和删除操作。

* 方法1. 使用私有属性对应的getter, setter, deleter装饰器

注意装饰器装饰的方法名将被作为属性来使用。

In [6]:
class Point2:
    def __init__(self, color):
        self.__yanse = color  # 私有属性
    @property
    def color(self):  # getter的名称不需要和私有属性名称一致
        return self.__yanse
    @color.setter
    def color(self, color):  # 注意函数的名称
        if isinstance(color,str):  self.__yanse = color
        else: print('color not accepted')

In [7]:
p2 = Point2('black')
print(p2.color)
p2.color = 'orange'
print(p2.color)

black
orange


In [8]:
p2.color = 3  # validation not passed
p2.color

color not accepted


'orange'

* 方法2. 调用property装饰器方法，设置fget, fset, fdel

用法：对象定义中增加一行attr_name = property(fget, fset, fdel)

In [9]:
class Point3:
    def __init__(self, color):
        self.__yanse = color  # 私有属性
    def get_color(self):
        return self.__yanse
    def set_color(self, color):
        if isinstance(color,str):  self.__yanse = color
        else: print('color not accepted')
    color = property(fget = get_color, fset = set_color)

In [10]:
p3 = Point3('red')
print(p3.color)
p3.color = 'blue'
print(p3.color)

red
blue


In [11]:
p3.color = 0xde
p3.color

color not accepted


'blue'

请思考：私有方法的作用是什么？什么时候需要私有方法？

回答：私有方法起到保护作用。当然，问题是保护什么？

In [12]:
class Access_Data:
    def __init__(self, data = 0):  self.__data = data
    def is_good(data):  return True
    def clean(data):  return data
    def __set_data(self, data):
        self.__data = data
    def set_data(self, data):
        if Access_Data.is_good(data): self.__set_data(data)
        else:  print('illegal data')
    def modify(self, data):
        data = Access_Data.clean(data)
        self.__set_data(data)  # we are sure the data is good

In [13]:
a1 = Access_Data(5)
a1.modify(6)
a1.set_data(6)

我们可以想象一个任务比较复杂，牵涉到若干子任务的重复执行，这些子任务本身可以写成方法，但是又不希望用户直接调用，因为这些方法只有在特定情形下才是合法的。

思考问题：

* 派生类是否继承了基类的私有属性？  
继承了所有的私有属性和私有方法    

* 派生类是否可以访问基类的私有属性？  
但是不能访问


## 类的特殊属性

以双下划线开始和结束的属性为类的特殊属性。

In [14]:
class Shape:
    cnt = 0
    def __init__(self, name):
        self.name = name
        self.area = 1
    def get_area(self):
        return self.area

In [15]:
Shape.__dict__  # 类对象的属性字典，注意类的属性和类的方法都在这个字典里

mappingproxy({'__dict__': <attribute '__dict__' of 'Shape' objects>,
              '__doc__': None,
              '__init__': <function __main__.Shape.__init__>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Shape' objects>,
              'cnt': 0,
              'get_area': <function __main__.Shape.get_area>})

In [16]:
s1 = Shape('circle')
s1.__dict__  # 实例对象的属性字典

{'area': 1, 'name': 'circle'}

In [17]:
class Circle(Shape):
    def __init__(self, d = 2):
        import math
        Shape.__init__(self, name = 'circle')
        self.diameter = d
        self.area = math.pi / 4 * d ** 2

In [18]:
Circle.__dict__  # 类对象的属性字典

mappingproxy({'__doc__': None,
              '__init__': <function __main__.Circle.__init__>,
              '__module__': '__main__'})

In [19]:
c1 = Circle(4)
c1.__dict__  #  实例对象的属性字典

{'area': 12.566370614359172, 'diameter': 4, 'name': 'circle'}

- 常见特殊属性
    * class 所属类
    * bases 基类元组
    * base 基类
    * name 名称
    * mro = method resolution order

In [20]:
print('实例所属类', c1.__class__)
print('基类元组', Circle.__bases__)
print('基类', Circle.__base__)
print('类的名称', Circle.__name__)
print('方法查找顺序', Circle.__mro__)

实例所属类 <class '__main__.Circle'>
基类元组 (<class '__main__.Shape'>,)
基类 <class '__main__.Shape'>
类的名称 Circle
方法查找顺序 (<class '__main__.Circle'>, <class '__main__.Shape'>, <class 'object'>)


### dict属性的应用

可以使用dict属性完成类成员的访问和修改

In [21]:
c1.__dict__['diameter'] = 10
print(c1.__dict__['diameter'], c1.__dict__['area'])  # of course, they do not match

10 12.566370614359172


In [22]:
del c1.__dict__['diameter']  # the attribute is gone
print(c1.__dict__)

{'name': 'circle', 'area': 12.566370614359172}


### 自定义属性

Python允许在类的定义外增加对象的属性，自定义属性，即类的定义体中不存在的属性。

In [23]:
class TestClass:
    def __init__(self, name = 'test'):
        self.name = name
t1 = TestClass()
t1.val = 1  # 通过实例对象创建一个自定义属性
t1.__dict__

{'name': 'test', 'val': 1}

In [24]:
t1.__dict__['key'] = 'sample101'  # 通过修改dict属性字典增加一个自定义属性
print(t1.key)

sample101


自定义属性的优点：自由，灵活

缺点：不在定义体中，难以追溯，不可预料的对象属性和行为

如何避免用户自定义属性？

### 拦截对于类成员的访问

如果不希望用户自定义属性，则需要在类定义中拦截对于成员的访问。

In [25]:
class BoldClass:
    def __init__(self, name = 'bold'):
        self.name = name
    def __setattr__(self, name, value):  #  拦截对于属性的设置操作
        attributes = {'name'}
        if name not in attributes:
            print('user defined attribute not allowed')
        else:  object.__setattr__(self, name, value)

In [26]:
b1 = BoldClass()
b1.val = 2
b1.__dict__

user defined attribute not allowed


{'name': 'bold'}

在用户设置对象属性时，进行属性值的合法性校验和数据清洗。

比如，Student类有属性name，name要求为字符串，首字母大写且要求去除前后空格。

In [1]:
class Student:
    def __init__(self, name):  self.name = name
    def __setattr__(self, name, value):
        print('access to attribute {0} is intercepted'.format(name))
        if name == 'name':
            if not isinstance(value, (str,bytes)):  #  检查是否为字符串或字节串
                print('only string allowed for attribute name')
            else:
                value = value.strip().capitalize()
                object.__setattr__(self, name, value)

In [2]:
s1 = Student('   tom  ')
s1.name

access to attribute name is intercepted


'Tom'

以上的例子中，设置实例属性的值既可以通过object对象的setattr方法，也可以通过修改实例对象的dict属性。前提是不把dict对象赋值为新的对象。

注意：__修改对象内容__ vs __绑定一个新对象__不是同一个概念。

In [3]:
print(s1.__dict__)
s1.__dict__['age'] = 20  # 使用实例的__dict__属性访问内部的键来修改对象内容，不会被拦截
s1.__dict__ = {'name' : 'Jack', 'age' : 21}  # 绑定一个新对象，这是一个设置操作，被拦截
print(s1.__dict__)

{'name': 'Tom'}
access to attribute __dict__ is intercepted
{'name': 'Tom', 'age': 20}


若重载了getattribute，则通过实例访问所有成员都会被拦截，包括dict属性。

此时不能使用dict属性来获取对应成员，会导致无限递归调用，形成死循环。

In [30]:
class GetAttr:
    done = False
    def __init__(self, name = 'test'):
        self.name = name
    def __getattribute__(self, name):
        print('access to {0} is intercepted'.format(name))
        return object.__getattribute__(self, name)  # 使用基类的getattribute来访问，注意不能使用self.__dict__[name]

In [31]:
g1 = GetAttr()
print(1, g1.name)  #  intercepted
print(2, g1.__dict__['name'])  #  intercepted too
print(3, g1.done)  #  intercepted too
print(4, GetAttr.done)  #  using class, not intercepted

access to name is intercepted
1 test
access to __dict__ is intercepted
2 test
access to done is intercepted
3 False
4 False


<h4>小结： setattr与getattribute的拦截操作区别</h4>  

**好像有点没看懂**  

* 修改实例的属性字典中的某个键值对时，未必需要重置dict对象，不会被setattr拦截
* 通过实例的属性字典获取某个键值对时，必须获取dict对象，一定会被getattribute拦截

## 类的特殊方法

类中的特殊方法以双下划线开头和结尾
* new方法，在创建对象时调用，返回当前对象的一个实例
* 构造方法init，创建完对象之后，对实例进行初始化
* 析构方法del，实现销毁实例对象时需要进行的收尾工作。

类的特殊方法通常与某种通用操作相关联，如下标化和python的内置函数，当使用内置函数操作时，系统会首先查找该类中对应的特殊方法。

对应于内置函数的特殊方法有：len, repr, str, bytes, format, bool, hash, dir等

In [32]:
class Items:
    def __init__(self, *it):
        self.content = it
    def __str__(self):
        return '+'.join([str(i) for i in self.content])
i1 = Items(1, 2, 3, 4, 5)
str(i1)

'1+2+3+4+5'

上面的例子重写了str函数，当使用内置函数str操作该对象时，会调用该对象的str方法。

下面的例子重写了repr函数。

In [33]:
class My_Dict:
    def __init__(self, **kwargs):
        self.content = kwargs
    def __repr__(self):
        f = lambda k,v: '{0}->{1}'.format(k,v) 
        k2v = [f(k,v) for (k,v) in self.content.items()]
        return ', '.join(k2v)

In [34]:
m1 = My_Dict(apple=  'red', banana = 'yellow', pear = 'white')
print(repr(m1))
# 与字典的原生repr方法比较
d1 = dict(apple=  'red', banana = 'yellow', pear = 'white')
print(repr(d1))

apple->red, banana->yellow, pear->white
{'apple': 'red', 'banana': 'yellow', 'pear': 'white'}


In [35]:
help(repr)

Help on built-in function repr in module builtins:

repr(obj, /)
    Return the canonical string representation of the object.
    
    For many object types, including most builtins, eval(repr(obj)) == obj.



可以发现：

内置对象的repr定义遵循一个规则： eval(repr(obj)) == obj.

### 运算符与运算的特殊方法

python中的运算符实际上也是通过调用对象的特殊方法实现的。如比较运算、算术运算、按位运算。

通过重写各运算符对应的特殊方法可以实现运算符的重载。

In [36]:
class MyList:
    def __init__(self, *data):  self.content = data
    def __len__(self):  return len(self.content)  # 重写了len函数
    def __add__(self, b):  # 重写了加法
        k = len(b) if len(b) < len(self) else len(self)
        content = [self.content[i] + b.content[i] for i in range(k)]
        return MyList(*content)  # note *content means multiple variables instead of one

In [37]:
m2 = MyList(1, 2, 3, 4, 5)
m3 = MyList(5, 7, 9, 2)
print((m2 + m3).content)

(6, 9, 12, 6)


### total_ordering 装饰器

进行大小比较需要实现多个特殊方法，比较繁琐。利用functools模块的total_ordering装饰器，可以简化代码。

只需要实现eq，另外再加上lt,le, ge, gt中的任意一个就可以实现完全比较。

In [38]:
import functools
@functools.total_ordering
class Cmp:
    def __init__(self, val):
        self.v = val
    def __eq__(self, other):
        return self.v == other.v
    def __lt__(self, other):
        return self.v < other.v

In [39]:
c1 = Cmp(5)
c2 = Cmp(9)
print(c1 > c2)
print(c1 <= c2)

False
True


## 方法的重载

重载的含义：在一个类中，如果有一系列同名方法，功能是相似的，但是形参个数和形参类型不同。在调用时，系统会根据函数参数决定调用哪一个。

作用：功能相似的操作只使用一个名称，更容易记忆和使用。

### 重载的实现方式

在C++和Java中，重载一般是通过定义多个同名但参数表不同的函数实现的。

在python中，函数的参数表已经体现了重载的思想，所以不需要定义多个同名函数。

Alert! 如果定义了多个同名函数，会导致前面定义的函数被最后一个函数定义覆盖。

通过重写对象的运算符函数可以实现运算符重载，即运算函数针对一种新的对象会执行一种新的操作，相当于方法的重载。

如加法本来定义在数值对象上，对于列表对象，重写add方法，就实现了加法运算符在列表对象上执行不同的操作。

In [40]:
class Addible_Dict:
    def __init__(self, **data):
        self.data = data
    def __add__(self, b):
        c = {}
        keys = set(self.data.keys()) | set(b.data.keys())
        for i in keys:
            c[i] = self.data.get(i, 0) + b.data.get(i, 0)
        return c

In [41]:
ad1 = Addible_Dict(a = 3, b = 4, c = 6)
ad2 = Addible_Dict(b = 100, a = 200, d = 300)
ad1 + ad2

{'a': 203, 'b': 104, 'c': 6, 'd': 300}

### 多态

多态的含义：不同对象对于同一个消息具有不同的响应。

C++和Java语言中，通过指向基类的指针，调用派生类的方法，实现接口通用但行为不同的目标。

Python语言中通过在派生类中重写基类的方法，实现同样的接口，使得派生类与基类的行为有所不同。从同一个基类派生的多个类会以不同的形式响应同一个消息。

### Python对于类型的处理：鸭子类型

python对于类型的控制并不像C++和Java那样严格，其中一个重要思路是：如果一只鸟走起来像鸭子，游起来像鸭子，叫起来像鸭子，那么这只鸟就可以称之为鸭子。

关注对象的操作，而非对象的实际类型。比如一个对象能够支持下标访问，则可以看作序列对象，不用管具体它是用数组实现的，还是用二叉树实现的。

In [42]:
class my_list:
    def __init__(self, data):  self.data = data
    def __next__(self):
        if self.data > 1:
            self.data -= 1
            return self.data
        else: raise StopIteration
    def __getitem__(self, idx):
        if idx < self.data:  return self.data - idx
    def __iter__(self):
        return self

In [43]:
for i in my_list(5):  # 这是一个什么类型的对象？
    print(i)
my_list(8)[3]

4
3
2
1


5

## 对象赋值与拷贝

* 数据与引用

数据在计算机中是存储在某段内存空间上的，对数据进行操作时需要引用(refer)该内存地址。大多数语言都使用变量来引用数据，使用变量进行操作时，实际上是对变量引用的数据进行操作。

但是在具体操作时，变量与引用的处理方式不同。


### 对象的引用与赋值

使用赋值语句时，实质是将右端(RHS)对象的引用传递给左端(LHS)变量。

In [44]:
a = 3
id(a)

499936800

In [45]:
a = a + 2
id(a)

499936864

解析：

* a = 3 建立整数类型对象3，将其引用传递给变量a

* a = a + 2 使用变量a找到对象3，使用其方法add(2)，得到一个新的对象，将新的对象的引用传递给变量a

注意python与C语言的区别：

在Python中，变量在python中只是一个名字，其类型是不固定的，取决于其指向的对象。改变变量的值，只是改变其引用，而不是改变其引用对象的值。

在C语言里，一个变量对应一个内存地址，其类型是确定的。改变变量的值，相当于修改该内存地址存储的值。

python是对象主导的，对象占据内存地址，决定内存地址的访问行为；C是变量主导的，决定内存地址和地址的访问行为。

In [46]:
b = 5  # 可以看到数据对象在python中是居于主导地位的
id(b), id(a)

(499936864, 499936864)

* 变量之间的赋值

如果将一个变量赋值给另一个变量时，会发生什么呢？

实际上系统会将右端变量的引用传递给左端变量，也就是说两个变量会指向同一个对象。

In [47]:
a = [1, 2, 3, 4]
b = a
id(a), id(b)

(93203912, 93203912)

这样，当我们使用其中一个变量引用该对象进行修改操作（如果是可变对象）时，另一个变量引用的对象也会被修改，因为它们指向的是同一个对象。

In [48]:
b.append('*')
a, b

([1, 2, 3, 4, '*'], [1, 2, 3, 4, '*'])

与C语言区分，下面这种方式并不是修改对象。

In [49]:
a = a + ['end']
a, b

([1, 2, 3, 4, '*', 'end'], [1, 2, 3, 4, '*'])

In [50]:
id(a), id(b)

(91840136, 93203912)

### 对象的浅拷贝

* 切片操作
* 对象实例化
* copy函数

In [51]:
obj_1 = [9, 5, 2, 7]
obj_2 = obj_1[:]
id(obj_1), id(obj_2)

(97836936, 97730632)

In [52]:
obj_2[2] = 'two'
obj_1, obj_2

([9, 5, 2, 7], [9, 5, 'two', 7])

In [53]:
obj_3 = list(obj_1)
obj_3[2] = 'three'
print(obj_1, obj_3)
id(obj_1), id(obj_3)

[9, 5, 2, 7] [9, 5, 'three', 7]


(97836936, 97607240)

In [54]:
obj_4 = obj_1.copy()
obj_4[2] = 'four'
print(obj_1, obj_4)
id(obj_1), id(obj_4)

[9, 5, 2, 7] [9, 5, 'four', 7]


(97836936, 93023240)

In [55]:
import copy
obj_5 = copy.copy(obj_1)
obj_5[2] = 'five'
print(obj_1, obj_5)
id(obj_1), id(obj_5)

[9, 5, 2, 7] [9, 5, 'five', 7]


(97836936, 92116744)

* 浅拷贝的问题

通过以上的操作我们得到了对象的浅拷贝，该对象的第一层元素被复制了，但是若该元素为复合数据，仅仅复制其引用。这就带来新的问题。

In [56]:
obj_c1 = [1, 2, [3, 4], [5, 6, 7]]
obj_c2 = obj_c1.copy()
id(obj_c2[2]), id(obj_c1[2])  # 注意两个对象的子对象指向同一个对象

(97605704, 97605704)

In [57]:
obj_c2[3] = ['ok']  # safe to modify the reference
obj_c1, obj_c2

([1, 2, [3, 4], [5, 6, 7]], [1, 2, [3, 4], ['ok']])

In [58]:
obj_c2[2].append('*')  # not safe to modify the object
obj_c1, obj_c2

([1, 2, [3, 4, '*'], [5, 6, 7]], [1, 2, [3, 4, '*'], ['ok']])

### 对象的深拷贝

使用copy模块的deepcopy方法，可以得到对象的深拷贝。此时，对象中的子对象会被递归复制。


In [59]:
import copy
obj_c1 = [1, 2, [3, 4], [5, 6, 7]]
obj_c3 = copy.deepcopy(obj_c1)
id(obj_c2[2]), id(obj_c3[2])  # not the same object

(97605704, 97661192)

In [60]:
obj_c3[2].extend('abc')
obj_c1, obj_c3

([1, 2, [3, 4], [5, 6, 7]], [1, 2, [3, 4, 'a', 'b', 'c'], [5, 6, 7]])

## 面向对象的程序设计小结

- 基本特征：
    * 封装
    * 继承
    * 多态

* 封装

对客观事物进行抽象后，将其属性和相关操作封装为对象，定义访问对象的接口，隐藏内部实现。

作用：封装保证了对象的独立性，防止外部程序破坏对象的内部数据。同时，通过对象组织代码，代码的结构性更好。

意义：提高了程序的可维护性。

* 继承

通过在现有类的基础上派生新的类，既能使用现有类的功能，也能增加新的功能。

作用：在不重写原有类的情况下对已有类的功能进行扩展。避免了代码复制。

意义：提高了代码的可重用性，提高了程序的可维护性。

* 多态

派生类同时具有基类的属性和方法，也具有新类的属性和方法，因而呈现出多个有效类型。

作用：使得每个对象可以用自己的方式响应共同的消息。

意义：提高程序的可维护性。